In [1]:
import pandas as pd
import pathlib
import duckdb
import openpyxl 
from datetime import datetime
import traceback
from IPython.display import display, HTML

In [2]:
# ==============================================================================
# Bloco 1: CONFIGURAÇÃO E LEITURA DO ARQUIVO DE CONTROLE
# ==============================================================================

# --- CAMINHOS DE ENTRADA E SAÍDA ---
PASTA_STAGING = pathlib.Path("dados_catalogo") / "staging"
CAMINHO_PARQUETS = str(PASTA_STAGING / "*.parquet")
PASTA_PROCESSED = pathlib.Path("dados_catalogo") / "processed"
PASTA_PROCESSED.mkdir(exist_ok=True)

# Arquivo de controle que você deve criar e manter
ARQUIVO_CONTROLE_PLATAFORMAS = 'plataformas_controle.xlsx'

# Arquivos de saída
ARQUIVO_TIMELINE_CSV = PASTA_PROCESSED / 'relatorio_timeline_plataformas.csv'
ARQUIVO_CONSOLIDADO_FILTRADO = PASTA_PROCESSED / 'dados_consolidados_filtrados.parquet'


In [3]:
print("--- Etapa 2: Validação e Filtragem ---")

# --- Leitura da lista de plataformas do arquivo Excel ---
try:
    df_controle = pd.read_excel(ARQUIVO_CONTROLE_PLATAFORMAS)
    # Valida se as colunas esperadas existem
    if 'Plataforma' not in df_controle.columns or 'USAR' not in df_controle.columns:
        raise ValueError("O arquivo de controle deve conter as colunas 'Plataforma' e 'USAR'.")
   
    # Cria a lista de plataformas a serem mantidas    
    PLATAFORMAS_PARA_MANTER = df_controle[df_controle['USAR'].str.upper() == 'S']['Plataforma'].tolist()
    
    print(f"Arquivo de controle '{ARQUIVO_CONTROLE_PLATAFORMAS}' lido com sucesso.")
    print(f"{len(PLATAFORMAS_PARA_MANTER)} plataformas serão mantidas no processamento.")
    plataformas_controle_ok = True
except Exception as e:
    print(f"ERRO ao ler o arquivo de controle: {e}")
    plataformas_controle_ok = False


--- Etapa 2: Validação e Filtragem ---
Arquivo de controle 'plataformas_controle.xlsx' lido com sucesso.
114 plataformas serão mantidas no processamento.


In [4]:
# ==============================================================================
# Bloco 2: GERAÇÃO DO RELATÓRIO COMPLETO DE VALIDAÇÃO (VERSÃO FINAL CORRIGIDA)
# ==============================================================================

if plataformas_controle_ok:
    print("\nIniciando a geração do relatório de validação...")
    try:
        con = duckdb.connect(database=':memory:')
        
        # 1. Gera a tabela de linha do tempo completa
        query_timeline = f"""
        SELECT data_ref, Platform_Name, COUNT(*) as total_registros
        FROM read_parquet('{CAMINHO_PARQUETS}', union_by_name=true)
        GROUP BY data_ref, Platform_Name
        """
        df_timeline_raw = con.execute(query_timeline).fetch_df()
        df_timeline_pivot = df_timeline_raw.pivot_table(
            index='Platform_Name', columns='data_ref', values='total_registros'
        ).fillna(0).astype(int)
        
        df_timeline_pivot = df_timeline_pivot.sort_index(axis=1)
        
        # Salva CSV e XLSX com data/hora
        timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
        caminho_csv = PASTA_PROCESSED / f'relatorio_timeline_plataformas_{timestamp}.csv'
        caminho_xlsx = PASTA_PROCESSED / f'relatorio_timeline_plataformas_{timestamp}.xlsx'
        
        df_timeline_pivot.to_csv(caminho_csv, sep=';', encoding='utf-8-sig')
        df_timeline_pivot.to_excel(caminho_xlsx)
        print(f"✔ Tabela de timeline completa salva em CSV e XLSX com o timestamp: {timestamp}")

        # 2. Análise de Novidades e Mudanças (Cronológica e Visual)
        print("\n" + "="*60)
        print("--- RELATÓRIO CRONOLÓGICO DE MUDANÇAS E NOVIDADES ---")
        print("="*60)
        
        plataformas_nos_dados = set(df_timeline_pivot.index)
        novas_plataformas = plataformas_nos_dados - set(df_controle['Plataforma'])
        if novas_plataformas:
            display(HTML("<h3>[!] NOVAS PLATAFORMAS ENCONTRADAS (não estão no seu arquivo de controle)</h3>"))
            df_novas = pd.DataFrame(sorted(list(novas_plataformas)), columns=['Plataforma'])
            display(df_novas.style.hide(axis="index"))
        else:
            display(HTML("<h4>✔ Nenhuma plataforma nova (fora do arquivo de controle) foi detectada.</h4>"))

        # 3. Análise de Variação de Catálogo semana a semana
        for i in range(len(df_timeline_pivot.columns)):
            semana_atual_str = df_timeline_pivot.columns[i]
            semana_atual_dt = pd.to_datetime(semana_atual_str)
            
            display(HTML(f"<br><h2>Semana {semana_atual_dt.strftime('%d/%m/%Y')}</h2>"))

            if i == 0:
                print("Esta é a primeira semana do histórico. Os dados abaixo servem como base inicial.")
                base_inicial = df_timeline_pivot[[semana_atual_str]].reset_index()
                base_inicial = base_inicial[base_inicial[semana_atual_str] > 0]
                base_inicial.columns = ['Plataforma', 'Registros']
                display(base_inicial.sort_values(by='Registros', ascending=False)
                        .style.format({'Registros': '{:,}'}).hide(axis="index"))
                display(HTML("<hr>"))
                continue

            semana_anterior_str = df_timeline_pivot.columns[i-1]
            entradas, saidas, gaps, variacoes = [], [], [], []

            for plataforma in PLATAFORMAS_PARA_MANTER:
                if plataforma not in df_timeline_pivot.index: continue
                registros_anteriores = df_timeline_pivot.loc[plataforma, semana_anterior_str]
                registros_atuais = df_timeline_pivot.loc[plataforma, semana_atual_str]
                
                if registros_atuais > 0 and registros_anteriores == 0:
                    if df_timeline_pivot.loc[plataforma, :semana_anterior_str].sum() > 0:
                        gaps.append({'Plataforma': plataforma, 'Registros Atuais': registros_atuais})
                    else:
                        entradas.append({'Plataforma': plataforma, 'Registros': registros_atuais})
                elif registros_anteriores > 0 and registros_atuais == 0:
                    saidas.append({'Plataforma': plataforma, 'Registros Anteriores': registros_anteriores})
                elif registros_anteriores > 0 and registros_atuais > 0:
                    variacao = (registros_atuais - registros_anteriores) / registros_anteriores # <-- LINHA CORRIGIDA
                    if abs(variacao) > 0.05:
                        variacoes.append({'Plataforma': plataforma, 'Variação': variacao, 'Registros Anteriores': registros_anteriores, 'Registros Atuais': registros_atuais})
            
            if not any([entradas, saidas, gaps]):
                print("Não houve variação no total de plataformas cobertas.")
            
            if entradas:
                display(HTML("<h4 style='color: #2E8B57;'>Novas Plataformas Cobertas</h4>"))
                df_entradas = pd.DataFrame(entradas).sort_values(by='Registros', ascending=False)
                display(df_entradas.style.format({'Registros': '{:,}'}).bar(subset=['Registros'], color='#90EE90').hide(axis="index"))

            if saidas:
                display(HTML("<h4 style='color: #CD5C5C;'>Plataformas que deixaram a cobertura</h4>"))
                df_saidas = pd.DataFrame(saidas).sort_values(by='Registros Anteriores', ascending=False)
                display(df_saidas.style.format({'Registros Anteriores': '{:,}'}).bar(subset=['Registros Anteriores'], color='#FFA07A').hide(axis="index"))

            if gaps:
                display(HTML("<h4 style='color: #DAA520;'>Plataformas que Retornaram à Cobertura</h4>"))
                df_gaps = pd.DataFrame(gaps).sort_values(by='Registros Atuais', ascending=False)
                display(df_gaps.style.format({'Registros Atuais': '{:,}'}).bar(subset=['Registros Atuais'], color='#FFD700').hide(axis="index"))

            if variacoes:
                display(HTML("<h4>Variações >5% no Catálogo de Conteúdos</h4>"))
                df_variacoes = pd.DataFrame(variacoes).sort_values(by='Variação', key=abs, ascending=False)
                display(df_variacoes.style.format({'Variação': '{:+.1%}', 'Registros Anteriores': '{:,}', 'Registros Atuais': '{:,}'}).background_gradient(subset=['Variação'], cmap='RdYlGn', vmin=-1, vmax=1).hide(axis="index"))

            display(HTML("<hr>")) 

    except Exception as e:
        print(f"ERRO ao gerar o relatório de validação: {e}")
        traceback.print_exc()


Iniciando a geração do relatório de validação...
✔ Tabela de timeline completa salva em CSV e XLSX com o timestamp: 2025-09-18_174118

--- RELATÓRIO CRONOLÓGICO DE MUDANÇAS E NOVIDADES ---


Esta é a primeira semana do histórico. Os dados abaixo servem como base inicial.


Plataforma,Registros
Archivio Luce,"68,820"
Claro TV+,"42,986"
NBA League Pass,"27,773"
Amazon Prime Video,"26,612"
Vivo Play,"25,107"
AppleTV,"10,772"
Sky+,"10,696"
Plex,"10,593"
Netflix,"7,348"
UNIVER VIDEO,"6,285"


Plataforma,Registros
YouTube,"8,695"
Tamanduá TV,"2,347"
Curta!On,995
Filmicca,391
Cardume,235
Cine Humberto Mauro Mais,160


Plataforma,Variação,Registros Anteriores,Registros Atuais
KweliTV,-45.5%,"1,403",764


...

Plataforma,Variação,Registros Anteriores,Registros Atuais
Simply South,+100.7%,"1,723","3,458"
IWantTFC,-75.0%,"2,252",564
Itaú Cultural Play,+11.0%,246,273


Não houve variação no total de plataformas cobertas.


Plataforma,Variação,Registros Anteriores,Registros Atuais
SBT Videos,-63.8%,442,160
Simply South,-49.9%,"3,458","1,732"
Itaú Cultural Play,+9.2%,273,298
TV Brasil Play,+6.6%,290,309


Não houve variação no total de plataformas cobertas.


Plataforma,Variação,Registros Anteriores,Registros Atuais
KINOA.TV,-49.7%,153,77
Canela.TV,+8.2%,"2,306","2,496"
TV Brasil Play,-5.8%,309,291


In [5]:
# ==============================================================================
# Bloco 3: CONSOLIDAÇÃO E FILTRAGEM FINAL (VERSÃO OTIMIZADA PARA MEMÓRIA)
# ==============================================================================
if plataformas_controle_ok:
    print("\n" + "="*60)
    print("--- CONSOLIDAÇÃO E FILTRAGEM FINAL (MODO OTIMIZADO) ---")
    print("="*60)
    
    con = None
    try:
        con = duckdb.connect(database=':memory:')
        
        # Converte a lista de plataformas para um formato que o SQL entende (ex: ('Netflix', 'Max', ...))
        plataformas_sql = tuple(PLATAFORMAS_PARA_MANTER)
        
        print("Consolidando e filtrando dados diretamente no banco de dados...")

        # Cria uma query que já filtra e se prepara para salvar
        # A cláusula WHERE aplica o filtro de plataformas diretamente no DuckDB
        # A cláusula COPY salva o resultado sem passar por um DataFrame gigante no Pandas
        query_filtragem_e_salvamento = f"""
        COPY (
            SELECT *
            FROM read_parquet('{CAMINHO_PARQUETS}', union_by_name=true)
            WHERE Platform_Name IN {plataformas_sql}
        ) TO '{ARQUIVO_CONSOLIDADO_FILTRADO}' (FORMAT PARQUET, COMPRESSION 'snappy');
        """
        
        # Executa a operação
        con.execute(query_filtragem_e_salvamento)
        
        print(f"✔ Arquivo consolidado e filtrado salvo em: '{ARQUIVO_CONSOLIDADO_FILTRADO}'")
        print("\nProcesso concluído com sucesso com baixo uso de memória!")

    except Exception as e:
        print(f"ERRO ao consolidar e filtrar os dados: {e}")
    
    finally:
        if con:
            con.close()


--- CONSOLIDAÇÃO E FILTRAGEM FINAL (MODO OTIMIZADO) ---
Consolidando e filtrando dados diretamente no banco de dados...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✔ Arquivo consolidado e filtrado salvo em: 'dados_catalogo\processed\dados_consolidados_filtrados.parquet'

Processo concluído com sucesso com baixo uso de memória!
